# Airports around

By last, I will filter the remaining candidates by its proximity to any airport or airports. In order to accomplish that goal, I will use the Lufthansa Public API, which has a functionality that gives you the five closest airports of a given coordinate and its distance in kilometers. Let's perform the operation

In [76]:
from dotenv import load_dotenv
from bson import ObjectId
import json, requests
from statistics import mean

In [2]:
from pymongo import MongoClient
client = MongoClient()
db = client.get_database('companies_project')

In [3]:
candidates = list(db.companies_usa_filt4.find({},{'coord':1}))

In [5]:
load_dotenv()
lufthansa_id = os.getenv('lufthansa_id')
lufthansa_secret = os.getenv('lufthansa_secret')

In [47]:
for cand in candidates[1:]:
    latitude = cand['coord']['coordinates'][1]
    longitude = cand['coord']['coordinates'][0]
    id = ObjectId(cand['_id'])
    
    request = 'https://api.lufthansa.com/v1/oauth/token'
    header_auth = {
                'client_id' : f'{lufthansa_id}',
                'client_secret': f'{lufthansa_secret}', 
                'grant_type' : 'client_credentials'}      

    r = requests.post(request, data = header_auth)
    j = r.json()
    token = j['access_token']
    header = { 'Authorization': 'Bearer ' + str(token), 'Accept': 'application/json'}

    resp = requests.get(f'https://api.lufthansa.com/v1/references/airports/nearest/{latitude},{longitude}?lang=en', headers=header)
    data_air = json.loads(resp.text)

    data_format = {}
    index = 0
    for airp in data_air['NearestAirportResource']['Airports']['Airport']:
        airp['Names']['Name']['name'] = airp['Names']['Name']['$']
        del airp['Names']['Name']['$']
        
        data_format[str(index)] = airp
        index += 1
    
    db.companies_usa_filt4.update_one({'_id':id}, {'$set':{'offices.airports': data_format}})

Now all our candidates have listed its five closest airports. Let's extract the median distance in kilometers for all of them, so we can rank the candidates.

In [48]:
airports = list(db.companies_usa_filt4.find({}, {'offices.airports':1}))

In [80]:
for airp in airports:
    id = ObjectId(airp['_id'])
    mean_dist = []
    for a in airp['offices']['airports']:
        mean_dist.append(airp['offices']['airports'][a]['Distance']['Value'])
    mean_add = mean(mean_dist)
    db.companies_usa_filt4.update_one({'_id':id}, {'$set':{'offices.airports.mean_dist': mean_add}})

In [95]:
dist_to_airp = list(db.companies_usa_filt4.find({}, {'offices.airports.mean_dist':1}))

In [98]:
unique_val = set([ap['offices']['airports']['mean_dist'] for ap in dist_to_airp])
unique_val

{42.2, 42.4, 42.6, 42.8, 43, 45.4, 60.6, 60.8, 61, 61.2, 86.6, 88.8}

There are mean values from 42 kilometers to 89. Let's take those which mean distance to its five airports is less than or equal to 42.5 kilometers.

In [110]:
offices_5 = list(db.companies_usa_filt4.find({'offices.airports.mean_dist':{'$lte':42.5}}))
[off['offices']['city'] for off in offices_5]

['Seattle', 'New York', 'New York']

After apply the last filter I end up with three final candidates: two of them in New York, and one in Seattle. Let's take them to a last collection and show the results in a map.

In [111]:
db.companies_usa_filt5.insert_many(offices_5)